# API Request Example to VRR
Define the name of the API endpoint you want to query and the parameters you need to pass.

In [7]:
# -*- coding: utf-8 -*-
# imports
import requests
import pandas as pd
import uuid
import time
import os
from datetime import datetime


In [ ]:
# add api request function
# https://efa.vrr.de/standard/



# Build the request parameters as a dictionary

def full_api_request(datetime_dt, place_dm, name_dm):
    params = {
        "language": "de",
        "mode": "direct",
        "outputFormat": "JSON",
        "type_dm": "stop",
        "useProxFootSearch": 0,
        "useRealtime": 1,
        "itdDateDay": datetime_dt.day,
        "itdDateMonth": datetime_dt.month,
        "itdDateYear": datetime_dt.year,
        "itdTimeHour": datetime_dt.hour,
        "itdTimeMinute": datetime_dt.minute,
        "place_dm": place_dm,
        "name_dm": name_dm,  # Replace with the desired stop name
    }
    
    # Define the API URL
    API_URL = "https://efa.vrr.de/standard/XML_DM_REQUEST"

    # Send the request to the API
    response = requests.get(API_URL, params=params)

    # Check if the request was successful
    if response.status_code == 200:
        # Parse the JSON response with utf-8 encoding
        data = response.json()
        print(data)
    else:
        print(f"Error: {response.status_code} - {response.text}")

    def make_uid(stop, scheduled_time, line):
        base = f"{stop}|{scheduled_time}|{line}"
        return str(uuid.uuid5(uuid.NAMESPACE_DNS, base))

    # Extract stops, delay times, and scheduled departures from the 'data' variable
    departures = data.get('departureList', [])

    results = []
    for dep in departures:
        # Extract relevant information from each departure
        stop_name = dep.get('stopName')
        platform = dep.get('platformName', dep.get('platform'))
        scheduled = dep.get('dateTime', {})
        scheduled_time = f"{scheduled.get('hour', '')}:{str(scheduled.get('minute', '')).zfill(2)}"
        real = dep.get('realDateTime', {})
        real_time = f"{real.get('hour', '')}:{str(real.get('minute', '')).zfill(2)}" if real else None
        line = dep.get('servingLine', {}).get('number')
        direction = dep.get('servingLine', {}).get('direction')
        delay = dep.get('servingLine', {}).get('delay')
        cancelled = dep.get('servingLine', {}).get('cancelled')
        connection_exists = not (str(cancelled) == "1")

        # Create a unique identifier for the departure
        uid = make_uid(stop_name, scheduled_time, line)

        results.append({
            'uuid': uid,
            'stop': stop_name,
            'platform': platform,
            'line': line,
            'direction': direction,
            'scheduled_departure': scheduled_time,
            'real_departure': real_time,
            'delay_min': int(delay) if delay not in (None, '', '-9999') else None,
            'connection_exists': connection_exists
        })

    df_departures = pd.DataFrame(results)
    print(f"Found {len(df_departures)} departures.")

    # make sure utf-8 encoding is used - vomit on sight :)
    lut = {"Ã¼": "ü", "Ã¶": "ö", "Ã¤": "ä", "ÃŸ": "ß", "Ã": "ß"}
    for col in ['stop', 'direction']:
        df_departures[col] = df_departures[col].replace(lut, regex=True)
    return df_departures, response.status_code

In [ ]:
# Define the datetime for the request
return
datetime_dt = datetime.now()

# Define the place and name for the stop
place_dm = "Gelsenkirchen"
name_dm = "HBF"

placename_list = [("Duisburg", "HBF"), ("Mönchengladbach", "HBF"), ("Wuppertal", "HBF"), ("Bochum", "HBF"), ("Dortmund", "HBF"), ("Essen", "HBF"), ("Düsseldorf", "HBF")]
placename_list = [("Duisburg", "HBF")]

final_df = pd.DataFrame()
# Make the API request and get the DataFrame
for place_dm, name_dm in placename_list:
    print(f"Requesting data for {place_dm} - {name_dm}")
    df, status_code = full_api_request(datetime_dt, place_dm, name_dm)
    if not df.empty:
        final_df = pd.concat([final_df, df], ignore_index=True)

# Display the final DataFrame
final_df

In [ ]:
delay_min = 0.5
placename_list = [("Duisburg", "HBF"), ("Mönchengladbach", "HBF"), ("Wuppertal", "HBF"), ("Bochum", "HBF"), ("Dortmund", "HBF"), ("Essen", "HBF"), ("Düsseldorf", "HBF")]


# calculate delay in milliseconds
total_requests = len(placename_list)
delay_s = delay_min * 60  # convert minutes to milliseconds
request_delay = delay_s / total_requests # time the actual requests so that they space out over the delay time

# initialize csv
if os.path.exists('final_departures.csv'):
    print("CSV file already exists, loading existing data...")
else:
    df = pd.DataFrame(columns=[
        'uuid', 'stop', 'platform', 'line', 'direction',
        'scheduled_departure', 'real_departure', 'delay_min', 'connection_exists'
    ])
    df.to_csv('final_departures.csv', index=False)
    print("Initialized CSV file")

# Main loop
print(f"Total requests: {total_requests}, Delay per request: {round(request_delay/60, 2)} minutes.")
print("Starting the request loop...")

# Load existing UUIDs only once at the start
try:
    existing_df = pd.read_csv('final_departures.csv', usecols=['uuid'])
    existing_uuids = set(existing_df['uuid'].dropna().astype(str))
    print(f"Loaded {len(existing_uuids)} existing UUIDs.")
except FileNotFoundError:
    existing_uuids = set()
    print("No existing UUIDs found, starting fresh.")

while True:
    print("Starting a new cycle of requests...")
    datetime_dt = datetime.now()

    for place_dm, name_dm in placename_list:
        df, status_code = full_api_request(datetime_dt, place_dm, name_dm)

        if not df.empty:
            df['uuid'] = df['uuid'].astype(str)
            new_df = df[~df['uuid'].isin(existing_uuids)]

            if not new_df.empty:
                new_df.to_csv('final_departures.csv', mode='a', header=not existing_uuids, index=False)
                existing_uuids.update(new_df['uuid'])
                print(f"Appended {len(new_df)} new departures. Status code: {status_code}")
            else:
                print("No new UUIDs to append.")
        else:
            print(f"No new departures found for {place_dm} - {name_dm}. Status code: {status_code}")

        print(f"Sleeping for {round(request_delay/60, 2)} minutes.")
        time.sleep(request_delay)

    print("Next cycle...")

CSV file already exists, loading existing data...
Total requests: 7, Delay per request: 0.07 minutes.
Starting the request loop...
Loaded 6094 existing UUIDs.
Starting a new cycle of requests...
{'parameters': [{'name': 'serverID', 'value': 'EFA-X-DELFI07_'}, {'name': 'requestID', 'value': '0'}, {'name': 'sessionID', 'value': '0'}, {'name': 'calcTime', 'value': '205.009'}, {'name': 'serverTime', 'value': '2025-07-12T11:28:12'}, {'name': 'logRequestId', 'value': '6c8a4afcd52d433a'}], 'dm': {'input': {'input': 'HBF'}, 'points': {'point': {'usage': 'dm', 'type': 'stop', 'name': 'Duisburg, Duisburg Hbf', 'stateless': '20016032', 'tariffArea': '', 'tariffAreaName': '', 'tariffLayer1': '', 'tariffLayer2': '', 'ref': {'id': '20016032', 'gid': 'de:05112:16032', 'omc': '5112000', 'placeID': '20', 'place': 'Duisburg', 'coords': '754137.00000,5302208.00000'}, 'infos': None}}}, 'arr': {'input': {'input': ''}, 'points': None}, 'dateTime': {'deparr': 'dep', 'ttpFrom': '20250401', 'ttpTo': '20251231'